In [2]:
%pylab inline
%load_ext autoreload
%autoreload 2
%load_ext Cython

from InstanceVariables import InstanceVariables
import SimulationInstance
import SimulationInstanceVectorized

Populating the interactive namespace from numpy and matplotlib


Before looking at the results of each simulation instance, both vectorized and not, let us first define the variables used and their values for each. The following parameter values are those used in the Santos, Santos, Pacheco paper supporting material.

As well as this, let us define 4 of the main social norms investigated

In [14]:
SternJudging = [[1, 0 ],
                [0, 1]]
SimpleStanding = [[1, 1],
                  [0, 1]]
Shunning = [[1, 0],
            [0, 1]]
ImageScore = [[1, 1],
              [0, 0]]

Now a function containing all the predefined variables for use in testing:

In [15]:
def SSP(population_size, generations, socialnorm):
    runs = 1
    mutation_rate = np.power(10*population_size, -1)

    execution_error = 0.08
    reputation_assignment_error = 0.01
    private_assessment_error = 0.01
    reputation_update_probability = 1
    cost = 1
    benefit = 5
    SimulationInstance.simulate(runs, generations, population_size,
        mutation_rate, execution_error, reputation_assignment_error, private_assessment_error,
        reputation_update_probability, socialnorm,
        cost, benefit)

# Population optimization #
Now let's have a quick look at the speed difference in a population of 50.

In [15]:
%%timeit
SSP(50, 3*np.power(10,3), SternJudging, optimized=False)

Cooperation index: 0.5537725
Cooperation index: 0.0433275
Cooperation index: 0.78902
Cooperation index: 0.0344675
1 loop, best of 3: 4.73 s per loop


The best case was 4.73 seconds in the unoptimized simulation, now if we test the optimized simulation.

In [14]:
%%timeit
SSP(50, 3*np.power(10,3), SternJudging, optimized=True)

Cooperation index: 0.8683766666666667
Cooperation index: 0.8917341666666667
Cooperation index: 0.6453216666666667
Cooperation index: 0.8730383333333334
1 loop, best of 3: 2.48 s per loop


The best case was 2.48 seconds for the optimized simulation. Compared to the 4.73 seconds it took for the unoptimized, we have a almost twice as quick of a simulation. However before we say it is ONLY twice as quick. Let us give the example of a population of size 1000, 20 times larger than what we've tested. Given we are only testing these with 3000 generations compared to the 300 000 generations required for the simulation to be accurate, it is simply to measure the speedup.

In [18]:
%%timeit
SSP(1000, 3*np.power(10,3), SternJudging, optimized=False)

Cooperation index: 0.3809449583333333
Cooperation index: 0.3568224583333333
Cooperation index: 0.409717
Cooperation index: 0.3925699166666667
1 loop, best of 3: 1min 29s per loop


As you can see, the amount of time the unoptimized program requires to run only 3000 generations with a population of 1000 is 1 minute and 29 seconds.

In [6]:
%%timeit
SSP(1000, 3*np.power(10,3), SternJudging, optimized=True)

Cooperation index: 0.3812939166666667
Cooperation index: 0.3980768333333333
Cooperation index: 0.38631845833333334
Cooperation index: 0.38385254166666666
1 loop, best of 3: 1.61 s per loop


Now for the optimized simulation, the time taken is only 1.61 seconds, which is extremely close to the time taken for a population of size 50. 

# Fitness Function #
Let's have a look at the speedup of the fitness function, responsible for determining the average fitness of an agent X against a tournament of other agents.

In [40]:
runs = 1
generations = 3*np.power(10, 3)
population_size = 10
socialnorm = [[0, 0], [0, 1]]
mutation_rate = np.power(10*population_size, -1)

execution_error = 0.08
reputation_assignment_error = 0.01
private_assessment_error = 0.01
reputation_update_probability = 1
cost = 1
benefit = 5
variables = InstanceVariables(runs, generations, population_size, mutation_rate,
                              execution_error, reputation_assignment_error,
                              private_assessment_error, reputation_update_probability,
                              socialnorm, cost, benefit)

Test the fitness function for small population sizes and then for larger population sizes

In [41]:
%%timeit
for _ in range(population_size):
    SimulationInstance.fitness_function(0, 1, variables)

100 loops, best of 3: 11.7 ms per loop


In [42]:
population_size = 50
variables = InstanceVariables(runs, generations, population_size, mutation_rate,
                              execution_error, reputation_assignment_error,
                              private_assessment_error, reputation_update_probability,
                              socialnorm, cost, benefit)

In [43]:
%%timeit
for _ in range(population_size):
    SimulationInstance.fitness_function(0, 1, variables)

1 loop, best of 3: 231 ms per loop


In [47]:
population_size = 100
variables = InstanceVariables(runs, generations, population_size, mutation_rate,
                              execution_error, reputation_assignment_error,
                              private_assessment_error, reputation_update_probability,
                              socialnorm, cost, benefit)

In [48]:
%%timeit
for _ in range(population_size):
    SimulationInstance.fitness_function(0, 1, variables)

1 loop, best of 3: 907 ms per loop


In [3]:
%%cython
import numpy as np
cimport numpy as np
cdef float payoff(int x, int y):
    cdef float return_payoff
    for _ in range(100):
        return_payoff += float(x + y)/2.0
    return return_payoff

DistutilsPlatformError: Unable to find vcvarsall.bat